In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import Voronoi
from shapely.geometry import MultiPoint, Point, Polygon, LineString
from shapely.ops import cascaded_union, polygonize
import fiona
import os
import json
from IPython.display import HTML
import folium
from folium.folium import Map
from folium import IFrame
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings('ignore')

In [2]:
def azimuth(point1, point2):
    '''azimuth between 2 shapely points (interval 0 - 360)'''
    angle = np.arctan2(point2.x - point1.x, point2.y - point1.y)
    return np.degrees(angle) if angle>0 else np.degrees(angle) + 360

In [3]:
data_file_path = r'C:\Users\Tal bauman\PycharmProjects\project1\dataDay.csv'
df = pd.read_csv(data_file_path)
points_geometry = [Point(p) for p in (list(zip(df.x, df.y)))]
gdf = gpd.GeoDataFrame(df, geometry=points_geometry)
points = (gdf[['x','y']]).as_matrix()
vor = Voronoi((df[['x','y']]).as_matrix())
lines = [LineString(vor.vertices[line]) for line in vor.ridge_vertices if -1 not in line]
geometry = [Polygon(poly) for poly in polygonize(lines)]
polygons_gdf = gpd.GeoDataFrame(columns=['Passed','Neighbors','Edge'], geometry= geometry)
pts = MultiPoint([Point(i) for i in points])
mask = pts.convex_hull
line_mask = LineString(mask.exterior.coords)
yMean = gdf["y"].mean()
xMean = gdf["x"].mean()
gdf = gdf.drop(columns=['x', 'y', 'urban_intersection'])

In [4]:
for j in range(len(polygons_gdf)):
    polpol=polygons_gdf.xs(j)['geometry']
    polygons_gdf.at[j,'geometry'] = polpol.intersection(mask)
    p = polygons_gdf.xs(j)['geometry']
    edge = False
    if p.intersects(line_mask):
        edge = True
    polygons_gdf.at[j,'Edge']  = edge
    polygons_gdf.at[j,'Passed']  = False


max_poly_edge = np.round(len(polygons_gdf.loc[polygons_gdf['Edge'] == True]) / 2)


In [5]:
polygons_gdf.crs = {'init' :'epsg:4326'}  
gdf.crs = {'init' :'epsg:4326'}  

polygons_gdf = gpd.sjoin(polygons_gdf,gdf , how='inner', op='intersects', lsuffix='left', rsuffix='right')
 
polygons_gdf['Value'] = polygons_gdf['id']
polygons_gdf=polygons_gdf.drop(columns=['id'])
polygons_gdf=polygons_gdf.drop(columns=['index_right'])


In [6]:
sj = gpd.sjoin(polygons_gdf,polygons_gdf , how='inner', op='intersects', lsuffix='left', rsuffix='right')
sj = sj.drop(columns=['Passed_right', 'Edge_right', 'Passed_right','Neighbors_right','Edge_left','Neighbors_left','Passed_left'])

for i in range(len(polygons_gdf)):
    polygons_gdf.at[i,'Neighbors']  = sj.xs(i)['index_right'].to_list()

    

In [280]:
Poly_gjson = sj['geometry'].to_json()
Polygons_j = folium.features.GeoJson(Poly_gjson)
mapa = folium.Map(location=[yMean, xMean],zoom_start=12.25)
mapa.add_child(Polygons_j)

In [ ]:
TH = 1000
max_distance = 1000 / 111030 #distance in deg

groops = []
solo = []
for i in range(len(polygons_gdf)):
    mainPolygon = polygons_gdf.xs(i)['geometry']
    mainPolygon_distance = 0
    mygroop=[]
    mygroop.append(i)
    poly_neighbors = polygons_gdf.xs(i)['Neighbors']
    if not polygons_gdf.xs(i)['Passed'] and len(poly_neighbors)>0:
        pp_value = []
        pp_value.append(polygons_gdf.xs(i)['Value'])
        counter=0
        while(np.sum(pp_value) < TH and counter < len(poly_neighbors)):
            poly_distance = mainPolygon.distance(polygons_gdf.xs(poly_neighbors[counter])['geometry'])
            if not polygons_gdf.xs(poly_neighbors[counter])['Passed'] and polygons_gdf.xs(poly_neighbors[counter])['Value']< TH and poly_distance < max_distance:
                pp_value.append(polygons_gdf.xs(poly_neighbors[counter])['Value'])
                polygons_gdf.xs(poly_neighbors[counter])['Passed'] = True
                mygroop.append(poly_neighbors[counter])
                poly_neighbors += polygons_gdf.xs(poly_neighbors[counter])['Neighbors']
                counter = 0
            else:
                counter+=1
        if len(mygroop)>1:
            groops.append(mygroop)

    if not polygons_gdf.xs(i)['Passed']:
        solo.append(i)


In [262]:
polygons2_gdf = gpd.GeoDataFrame(columns=['geometry','Value'], geometry='geometry')


for g in groops:
    if(len(g)>1):
        poly_val=[]
        v = polygons_gdf.xs(g[0])['geometry']
        for p in g:
            v = v.union(polygons_gdf.xs(p)['geometry'])
            poly_val.append(polygons_gdf.xs(p)['Value'])
        gdf2 = gpd.GeoDataFrame({'geometry': [v], 'Value': [np.sum(poly_val)]})
        polygons2_gdf = polygons2_gdf.append(gdf2, ignore_index=True)


for i in solo:
    gdf2 = gpd.GeoDataFrame({'geometry': [polygons_gdf.xs(i)['geometry']], 'Value': [polygons_gdf.xs(i)['Value']]})
    polygons2_gdf = polygons2_gdf.append(gdf2, ignore_index=True)


In [8]:
polygons_gdf

,Passed,Neighbors,Edge,geometry,Value
0,False,"[0, 4, 2]",True,"POLYGON ((34.75263308218913 32.03018551107306,...",1
1,False,"[2, 1, 24, 5, 8]",False,"POLYGON ((34.75646373148458 32.03115844882648,...",4
2,False,"[0, 4, 2, 1, 24, 124, 25]",True,"POLYGON ((34.75617812122093 32.03108590733629,...",1
3,False,"[5, 35, 78, 77, 3, 6, 76, 79]",False,"POLYGON ((34.7638871786633 32.03304391300335, ...",3
4,False,"[0, 4, 2, 124, 58]",True,"POLYGON ((34.7522854620773 32.03198004957269, ...",2
5,False,"[1, 5, 8, 3, 6, 7]",False,"POLYGON ((34.76255480471895 32.03270550642729,...",11
6,False,"[5, 3, 6, 76, 7, 80, 83, 82, 84, 86]",False,"POLYGON ((34.76390967539518 32.03347799406894,...",3
7,False,"[5, 8, 6, 7, 83, 126, 87]",False,"POLYGON ((34.7608997639493 32.03647986956365, ...",1
8,False,"[1, 24, 5, 8, 7, 126, 26]",False,"POLYGON ((34.75750388962687 32.03657624093821,...",1
9,False,"[12, 13, 9, 11, 10]",False,"POLYGON ((34.75611265862472 32.05637775873951,...",7


In [258]:
polygons2_gdf.crs = fiona.crs.from_epsg(4326)
Poly_gjson2 = polygons2_gdf.to_crs(epsg='4326').to_json()
Polygons_j2 = folium.features.GeoJson(Poly_gjson2,style_function=lambda x: {'fillColor': 'green' , 'color' : 'black'})
mapa.add_child(Polygons_j2)

IndexError: list index out of range

In [11]:
wms = folium.raster_layers.WmsTileLayer(url='http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}')
wms.add_to(mapa)
mapa

In [10]:
edges = polygons_gdf.loc[polygons_gdf['Edge'] == True]
edges = edges.reset_index(drop=True)
edges['azimuth'] = 0
center_point = Point(xMean,yMean)

for i in range(len(edges)):
    p = edges.xs(i)['geometry'].centroid
    az = np.round(azimuth(p,center_point))
    edges.at[i,'azimuth'] = az
edges = edges.sort_values(by=['azimuth'])
edges = edges.reset_index(drop=True)

In [13]:
groop_num = 10
groop_jump = int(max_poly_edge/groop_num)
polygons_gdf['Passed'] = False
edges['Groop'] = np.empty((len(edges), 0)).tolist()

while(len(polygons_gdf.loc[polygons_gdf['Passed'] == False])>0):
    for i in range(0,int(max_poly_edge),groop_jump):
        for n in edges.xs(i*2)['Neighbors']:
            if not polygons_gdf.xs(n)['Passed']:
                edges.xs(i*2)['Groop'].append(n)
                edges.xs(i*2)['Neighbors'] += polygons_gdf.xs(n)['Neighbors']
                polygons_gdf.at[n,'Passed'] = True
                break
                
edges_gdf = gpd.GeoDataFrame(columns=['geometry','Value'], geometry='geometry')


for g in range(len(edges)):
    if(len(edges.xs(g)['Groop'])>1):
        poly_val=[]
        v = edges.xs(g)['geometry']
        for p in edges.xs(g)['Groop']:
            v = v.union(polygons_gdf.xs(p)['geometry'])
            poly_val.append(polygons_gdf.xs(p)['Value'])
        gdf2 = gpd.GeoDataFrame({'geometry': [v], 'Value': [np.sum(poly_val)]})
        edges_gdf = edges_gdf.append(gdf2, ignore_index=True)
        
edges_gdf.crs = fiona.crs.from_epsg(4326)
edges_gjson = edges_gdf.to_crs(epsg='4326').to_json()
edges_j = folium.features.GeoJson(edges_gjson)
m = folium.Map(location=[yMean, xMean],zoom_start=14)
m.add_child(edges_j)